In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from db_utils import query_hive_ssh
import re
import copy

# Talk Page Diff Sampling

In this notebook we will be taking samples from the full set of user talk page diffs for annotation.

## Bot Filtering

We don't want to include messages from bots in our sample.

### Resources
- [overview](https://www.mediawiki.org/wiki/Manual:Bots)
- [users in bot group](https://en.wikipedia.org/w/index.php?title=Special:ListUsers/bot&offset=BrokenAnchorBot&group=bot)
- [user group table docs](https://www.mediawiki.org/wiki/Manual:User_groups_table)
- [recent chnages table docs](https://www.mediawiki.org/wiki/Manual:Recentchanges_table)

### Summary
There is no great way to determine bots. Here are 3 strategies:

1. The `recentchnages` table contains 30 days of data and tells you which revisions where made by bots. This is the most reliable data, but there may be a seasonal trend in harassment that biases the sample.

2. Check if the user is in the bot group in the `user group` table. Drawbacks are:
    - Group membership can change over time. There are many bot-flagged edits by users that are no longer in a user group providing the bot right. Likewise there are many edits not bot-flagged by users that now have the bot right (which they may or may not use for each edit).
    - Not all bots are a member of this group (there are other groups that provide this right, sysop, for example).
3. Remove users with 'bot|Bot|BOT' in the user_name. Almost all bots in the 'bot' user group fit this pattern, but there will be many false positives. It seems unlikely that these false positives result in biased sample ...

For now I will go with option 3.

## Data Cleaning

For now, we basically just strip markup and remove duplicate diffs.

##### Steps
1. remove dupilicate rev_ids
3. remove duplicate diffs
4. clean diffs
     - ignore diffs with no content added
     - replace 'NEWLINE' with '\n'
     - strip mw markup

In [2]:
months = ['January',
          'February',
          'March',
          'April',
          'June',
          'July',
          'August',
          'September',
          'October',
          'November',
          'December',
        ]


month_or = '|'.join(months)
date_p = re.compile('\d\d:\d\d, \d?\d (%s) \d\d\d\d \(UTC\)' % month_or)
    
def remove_date(comment):
    return re.sub(date_p , '', comment )



pre_sub_patterns = [
                    ('\[\[Image:.*?\]\]', ''),
                    ('<!-- {{blocked}} -->', ''),
                    ('NEWLINE', '\n'),
                    ('\[\[File:.*?\]\]', ''),
                    ('\[\[User:.*?\|.*?\]\]', ''),
                    ('\(\[\[User talk:.*?\|talk\]\]\)', ''),
                   ]

post_sub_patterns = [
                    ('--', ''),
                    (' :', ' '),
                    ]

def substitute_patterns(s, sub_patterns):
    for p, r in sub_patterns:
        s = re.sub(p, r, s)
    return s

def strip_html(s):
    s = BeautifulSoup(s, 'html.parser').get_text()
    return s


def clean(df):
    df = copy.deepcopy(df)
    df.rename(columns = {'insertion': 'diff'}, inplace = True)
    df.dropna(subset = ['diff'], inplace = True)
    df['clean_diff'] = df['diff']
    
    df['clean_diff'] = df['clean_diff'].apply(remove_date)
    df['clean_diff'] = df['clean_diff'].apply(lambda x: substitute_patterns(x, pre_sub_patterns))
    df['clean_diff'] = df['clean_diff'].apply(strip_mw)
    df['clean_diff'] = df['clean_diff'].apply(strip_html)
    df['clean_diff'] = df['clean_diff'].apply(lambda x: substitute_patterns(x, post_sub_patterns))


    try:
        del df['rank']
    except:
        pass
    df.dropna(subset = ['clean_diff'], inplace = True)
    df = df[df['clean_diff'] != '']
    return df

In [3]:
#show_comments(clean(df[:100]), 100)

## Admin Messages
### Administrative Message Types Observed

- responses to to valdalism 
- new user greetings
- 5 million articles celebration
- speedy deletion
- deletion
- unfree files
- article for creation:
- comments with embedded image tags that contain 'warning' or 'information'

In [4]:
def show_comments(d, n = 10):
    for i, r in d[:n].iterrows():
        print(r['diff'])
        print('_' * 80)
        print(r['clean_diff'])
        print('\n\n', '#' * 80, '\n\n')

def find_pattern(d, pattern, column):
    p = re.compile(pattern)
    return d[d[column].apply(lambda x: p.search(x) is not None)]

def exclude_pattern(d, pattern, column):
    p = re.compile(pattern)
    return d[ d[column].apply(lambda x: p.search(x) is None)]

def exclude_few_tokens(d, n):
    return d[d['clean_diff'].apply(lambda x: len(x.split(' ')) > n)]

def exclude_short_strings(d, n):
    return d[d['clean_diff'].apply(lambda x: len(x) > n)]  

def remove_admin(d, patterns):
    d_reduced = copy.deepcopy(d)
    for pattern in patterns:
        d_reduced = exclude_pattern(d_reduced, pattern, 'diff')
    return d_reduced

patterns =[
    '\[\[Image:Octagon-warning',
    '\[\[Image:Stop',
    '\[\[Image:Information.',
    '\[\[Image:Copyright-problem',
    '\[\[Image:Ambox',
    '\[\[Image:Broom',
    '\[\[File:Information',
    '\[\[File:AFC-Logo_Decline',
    '\[\[File:Ambox',
    '\[\[File:Nuvola',
    '\[\[File:Stop',
    '\[\[File:Copyright-problem',
    '\[\[File:Copyright-problem',
    '\[\[File:Copyright-problem',
    '\[\[File:Copyright-problem',
    '\|alt=Warning icon\]\]',
    'The article .* has been \[\[Wikipedia:Proposed deletion\|proposed for deletion\]\]',
    'Your submission at \[\[Wikipedia:Articles for creation\|Articles for creation\]\]',
    'A file that you uploaded or altered, .*, has been listed at \[\[Wikipedia:Possibly unfree files\]\]',
    'User:SuggestBot',
    '\[\[Wikipedia:Criteria for speedy deletion\|Speedy deletion\]\] nomination of',
    "Please stop your \[\[Wikipedia:Disruptive editing\|disruptive editing\]\]. If you continue to \[\[Wikipedia:Vandalism\|vandalize\]\] Wikipedia, as you did to .*, you may be \[\[Wikipedia:Blocking policy\|blocked from editing\]\]",
    "Hello.*and.*\[\[Project:Introduction\|welcome\]\].* to Wikipedia!",
    'Nomination of .* for deletion',
    '== Welcome to Wikipedia! ==',
    '== Welcome! ==',
    '== 5 Million: We celebrate your contribution ==',
    ]

In [5]:
#d['diff'].apply(lambda x: re.search('\[\[Image(.*?)\]\].*', x)).dropna().apply(lambda x:x.group(1)).value_counts().head(50)
#d['diff'].apply(lambda x: re.search('\[\[File:(.*?)\]\].*', x)).dropna().apply(lambda x:x.group(1)).value_counts().head(50)
#show_comments(find_pattern(d,'\|alt=Warning icon\]\]', 'diff' ), 100)

# Post Based Random Sample
Consider comments made since min_timestamp. Take n random comments from non-bot users.

In [83]:
n = 100000
min_timestamp = '2014-03-01T00:00:00Z'

In [84]:
query = """
SELECT 
    diffs.*
FROM
    enwiki.user_talk_diff_no_admin diffs
WHERE
    rev_timestamp > '%(min_timestamp)s'
    AND user_text != 'MediaWiki message delivery'
    AND user_text != 'Maintenance script'
    AND user_text NOT RLIKE 'bot|Bot|BOT'
ORDER BY RAND()
LIMIT %(n)d
"""

params = {
    'n': int(n * 1.5),
    'min_timestamp': min_timestamp
    }

df = query_hive_ssh(query % params, 'post_sample.tsv', priority = True, quoting=3, delete = False)
df.columns = [c.split('.')[1] for c in df.columns]

In [85]:
print('Raw:', df.shape[0])
clean_df = clean(df)
print('Cleaned: ', clean_df.shape[0])
reduced_df = remove_admin(clean_df, patterns)
print('No Admin', reduced_df.shape[0])
reduced_df = exclude_few_tokens(reduced_df, 3)
print('No Few Words: ', reduced_df.shape[0])
reduced_df = exclude_short_strings(reduced_df, 20)
print('No Few Chars: ', reduced_df.shape[0])

Raw: 150000
Cleaned:  126017
No Admin 126017
No Few Words:  103071
No Few Chars:  99464


In [86]:
reduced_df[:1000].to_csv('../data/1k_no_admin_post_sample.tsv', sep = '\t')
reduced_df[:n].to_csv('../data/%dk_no_admin_post_sample.tsv' % (int(n / 1000)), sep = '\t')

In [77]:
#show_comments(reduced_df, 100)

# User Based Random Sample

Consider comments made since min_timestamp. Take up to k random comments from each of n random non-bot users

In [5]:
k = 5
n = 10000
min_timestamp = '2014-03-01T00:00:00Z'

In [6]:
query = """
SELECT 
    revisions.*
FROM
    (SELECT
        a.*
        FROM
        (
        SELECT
            d.*,
            RANK() OVER (PARTITION BY d.user_id ORDER BY RAND()) as rank
        FROM
            enwiki.user_talk_diff d
        WHERE 
            rev_timestamp > '%(min_timestamp)s'
        ) a
    WHERE rank <= %(k)d
    )
    revisions

JOIN
    (
    SELECT
        a.*
    FROM (
        SELECT
            user_id,
            RAND() as key
        FROM
            enwiki.user_talk_diff d
        WHERE 
            rev_timestamp > '%(min_timestamp)s'
            AND user_text != 'MediaWiki message delivery'
            AND user_text != 'Maintenance script'
            AND user_text NOT RLIKE 'bot|Bot|BOT'
        GROUP BY user_id
        ) a
    ORDER BY key
    LIMIT %(n)d
    ) users
ON
    revisions.user_id = users.user_id
"""

params = {
    'k': k,
    'n': n,
    'min_timestamp': min_timestamp
    }

### DEPRECATED ###
#user_sample_df = query_hive_ssh(query % params, 'random_sample.tsv', priority = True, quoting=3)
#user_sample_df.columns = [c.split('.')[1] for c in user_sample_df.columns]

# Blocked Users
For n random user block events that occured after min_timestamp, take up to the k most recent comments before being blocked for harassment or personal attacks.

In [87]:
k = 25
n = 150000
min_timestamp = '2000-03-01T00:00:00Z'

In [88]:
query = """

SELECT
    revisions.*
FROM
    (
    SELECT 
        diffs.*,
        RANK() OVER (PARTITION BY events.key, events.user_text ORDER BY diffs.rev_timestamp DESC) as rank
    FROM
        (
        SELECT
            a.*
        FROM
            (
            SELECT 
              log_title as user_text,
              log_timestamp,
              RAND() as key
            FROM enwiki.logging 
            WHERE
              log_type = 'block'
              AND log_action = 'block'
              AND log_comment RLIKE 'harassment|personal attack'
              AND log_timestamp > '%(min_timestamp)s'
            ) a
        ORDER BY key
        LIMIT %(n)d
        ) events
    JOIN
        enwiki.user_talk_diff_no_admin diffs
    ON
        diffs.user_text = events.user_text
    WHERE
        diffs.rev_timestamp <= events.log_timestamp
    ) revisions
WHERE
    rank <= %(k)d
"""

params = {
    'k': k,
    'n': n,
    'min_timestamp': min_timestamp
    }


df = query_hive_ssh(query % params, 'blocked_user_random_sample.tsv', priority = True, delete = False, quoting=3)
df.columns = [c.split('.')[1] for c in df.columns]

In [89]:
print('Raw:', df.shape[0])
clean_df = clean(df)
print('Cleaned: ', clean_df.shape[0])
reduced_df = remove_admin(clean_df, patterns)
print('No Admin', reduced_df.shape[0])
reduced_df = exclude_few_tokens(reduced_df, 3)
print('No Few Words: ', reduced_df.shape[0])
reduced_df = exclude_short_strings(reduced_df, 20)
print('No Few Chars: ', reduced_df.shape[0])

Raw: 24262
Cleaned:  21103
No Admin 21103
No Few Words:  16946
No Few Chars:  16505


In [90]:
k = int(reduced_df.shape[0] / 1000.0)
k

16

In [91]:
reduced_df[:1000].to_csv('../data/1k_no_admin_blocked_user_post_sample.tsv', sep = '\t')
k = int(reduced_df.shape[0] / 1000.0)
reduced_df.to_csv('../data/%dk_no_admin_blocked_user_post_sample.tsv' %  k, sep = '\t')

# Blocked User Longitudinal Data
For some users who got blocked for harassment/personal attacks, get all their user talk 

##### Data QA

In [50]:
query = """
SELECT
    COUNT(DISTINCT(user_text)) as n
FROM
    enwiki.blocked_user
"""

d = query_hive_ssh(query, 'qa.tsv', priority = True)

In [51]:
print('Num Users Blocked for Harassment')
print(d['n'].iloc[0])

Num Users Blocked for Harassment
4140


In [22]:
tables = [
            ('revision', 'rev_user_text'),
            ('user_talk_revision', 'rev_user_text'),
            ('user_talk_diff', 'user_text'),
            ('user_talk_diff_no_admin', 'user_text'),
            ('blocked_user_talk_diff', 'user_text'),
            ('blocked_user_talk_diff_no_admin', 'user_text'),
    ]
query = """
SELECT
    b.user_text,
    COUNT(*) as n
FROM
    enwiki.%s t
JOIN
    enwiki.blocked_user b
ON
    b.user_text = t.%s
GROUP
    BY b.user_text
"""

for t in tables:
    print('Num Blocked Users with a revision in: ', t[0])
    d = query_hive_ssh(query % t, 'qa.tsv', priority = True)
    print(d.shape[0], d['n'].sum())

Num Blocked Users with a revision in:  revision
3745 4781051
Num Blocked Users with a revision in:  user_talk_revision
2891 589562
Num Blocked Users with a revision in:  user_talk_diff
2824 583755
Num Blocked Users with a revision in:  user_talk_diff_no_admin
2821 521879
Num Blocked Users with a revision in:  blocked_user_talk_diff
2824 607881
Num Blocked Users with a revision in:  blocked_user_talk_diff_no_admin
2821 541429


We have 4.1k users who were blocked for harassment, but only 3.7k have any revision and only 2.8k have a user talk revision. Whats going on?

In [194]:
query = """
SELECT
    b.user_text
FROM
    (
    SELECT
        DISTINCT(%(username)s) as user_text
    FROM
        enwiki.%(table)s 
    ) t
RIGHT JOIN
    enwiki.blocked_user b
ON
    t.user_text = b.user_text 
WHERE 
    t.user_text is NULL
"""
params = {
    'username': 'rev_user_text',
    'table': 'revision'
}
d = query_hive_ssh(query % params, 'qa.tsv', priority = True)

In [201]:
d.head()

,b.user_text
0,100.34.12.38
1,103.27.226.0/24
2,117.197.156.176
3,142.162.218.188
4,148.233.165.0/24


I manually checked a bunch of examples in the prod DB and they really don't have any revisions. In some cases, the event happened off wiki (e.g. AOL, MSN). In other cases, maybe the revisions were deleted?

#### Sampling
How many users can we sample?

In [6]:
query = """
SELECT
    user_text,
    COUNT(*) num_user_talk_revs
FROM
    enwiki.blocked_user_talk_diff_no_admin
GROUP
    BY user_text
"""

user_talk_revs_per_user_df = query_hive_ssh(query, 'random_sample.tsv', priority = True)


In [9]:
user_talk_revs_per_user_df['num_user_talk_revs'].value_counts().sort_index().head(10)

1     575
2     316
3     235
4     164
5     130
6     108
7      82
8      73
9      55
10     48
Name: num_user_talk_revs, dtype: int64

In [13]:
user_talk_revs_per_user_df['num_user_talk_revs'].value_counts().sort_index().tail(10)

11462    1
11820    1
12151    1
12799    1
13573    1
15660    1
18568    1
19782    1
22548    1
24568    1
Name: num_user_talk_revs, dtype: int64

In [29]:
k = 1000
print('Num Users: ', user_talk_revs_per_user_df.shape[0])
print('Num Comments: ', user_talk_revs_per_user_df['num_user_talk_revs'].sum())

print ('Num Users with < %d Edits: ' % k,  user_talk_revs_per_user_df[user_talk_revs_per_user_df['num_user_talk_revs'] < k].shape[0])
print('Num Comments from Users with  < %d Edits: ' % k, user_talk_revs_per_user_df[user_talk_revs_per_user_df['num_user_talk_revs'] < k]['num_user_talk_revs'].sum() )

Num Users:  2821
Num Comments:  521879
Num Users with < 1000 Edits:  2737
Num Comments from Users with  < 1000 Edits:  122091


In [92]:
query = """

SELECT
    diffs.rev_comment,
    SUBSTRING(diffs.insertion, 0, 1000) as insertion,
    diffs.insert_only,
    diffs.rev_id,
    diffs.page_id,
    diffs.page_title,
    diffs.rev_timestamp,
    diffs.user_id,
    diffs.user_text,
    diffs.block_reasons,
    diffs.block_timestamps,
    diffs.block_actions,
    diffs.block_params
FROM
    enwiki.blocked_user_talk_diff_no_admin diffs
JOIN
    (
    SELECT
        user_text
    FROM
        enwiki.blocked_user_talk_diff
    GROUP
        BY user_text
    HAVING 
        COUNT(*) < %(max_comments)d
    ) users
    
ON
    diffs.user_text = users.user_text
"""

params = {
    'max_comments': 1000
    }

try:
    df = query_hive_ssh(query % params, 'blocked_user_long.tsv', priority = True, delete = False, quoting=3)
except:
    pass

In [93]:
df.columns = [c.replace('diffs.', '') for c in df.columns]

In [94]:
print('Raw:', df.shape[0])
clean_df = clean(df)
print('Cleaned: ', clean_df.shape[0])
reduced_df = remove_admin(clean_df, patterns)
print('No Admin', reduced_df.shape[0])
reduced_df = exclude_few_tokens(reduced_df, 3)
print('No Few Words: ', reduced_df.shape[0])
reduced_df = exclude_short_strings(reduced_df, 20)
print('No Few Chars: ', reduced_df.shape[0])

Raw: 113934
Cleaned:  101278
No Admin 101278
No Few Words:  83505
No Few Chars:  81490


In [95]:
reduced_df.to_csv('../data/all_no_admin_blocked_user_long_sample.tsv', sep = '\t', index = False)

# Scratch

In [13]:
query = """
SELECT 
  REGEXP_REPLACE(REGEXP_REPLACE(log_title, '_', ' '), 'User:', '')  as user_text,
  log_comment reason,
  log_timestamp timestamp,
  log_params params
FROM enwiki.logging 
WHERE
  log_type = 'block'
  AND log_action = 'block'
  AND log_comment RLIKE 'harassment|personal attack'
"""

df = query_hive_ssh(query , 'block_events.tsv', priority = True, delete = False, quoting=3)

In [16]:
df.head

<bound method NDFrame.head of                                    user_text  \
0                            172.166.166.226   
1                            200.67.239.0/24   
2                            200.67.239.0/24   
3                             200.67.239.225   
4                            62.219.229.0/24   
5                               Trollminator   
6                           148.233.165.0/24   
7                            200.171.69.0/24   
8                            65.219.238.0/24   
9                             213.138.128.14   
10                             134.22.70.189   
11                              134.22.68.75   
12                              134.22.68.75   
13                             134.22.71.117   
14                              134.22.70.98   
15                                     Jewjg   
16                                 ThinkPink   
17                              Peter Phipps   
18                                NeverAgain   
19        

In [377]:
s1 = set(reduced_clean_blocked_user_long_df.index)
s2 = set(clean_blocked_user_long_df.index)
s3 = s2 - s1
d_excluded = clean_blocked_user_long_df.ix[s3]

In [61]:
#show_comments( find_pattern(d_excluded, patterns[0], 'diff'), 100)

In [2]:
sum([1, 1])

2